In [0]:
%sql

create schema if not exists rwd.bronze;
     

In [0]:
source_path = "/Volumes/rwd/bronze/myvolume/bronze/staging/observation/"
checkpoint_path = "/Volumes/rwd/bronze/myvolume/bronze/observation_raw/checkpoint/"
schema_location = "/Volumes/rwd/bronze/myvolume/bronze/observation_raw/schema/"


In [0]:
# Autoloader read
df = (spark.readStream
          .format("cloudFiles")
          .option("cloudFiles.format", "csv")
          .option("header", "true")
          .option("inferSchema", "true")
          .option("cloudFiles.maxFilesPerTrigger", 1)
          .option("cloudFiles.schemaLocation", schema_location)  
          .load(source_path)
     )

In [0]:
# Write Bronze table (append)
(df.drop("_rescued_data")
    .writeStream
    .format("delta")
    .option("checkpointLocation", checkpoint_path)
    .option("mergeSchema", "true")
    .outputMode("append")
    .trigger(availableNow=True)
    .toTable("rwd.bronze.observation_raw")
)

In [0]:
%sql

select * from rwd.bronze.observation_raw

Patient_ID,Observation,Doctor,Observation_Date
P001,Elevated tumor markers,Dr. Menon,2026-01-15
P002,No detectable lesions,Dr. Kapoor,2025-12-20
P003,Increase in lymph node size,Dr. Reddy,2026-01-10
P004,Lesion unchanged over 6 months,Dr. Sharma,2026-01-05
P005,New hepatic metastasis,Dr. Iyer,2026-01-18
P006,Clear PET scan,Dr. Banerjee,2025-11-30
P007,Multiple pulmonary nodules,Dr. Thomas,2026-01-22
P008,Minimal residual disease,Dr. Gupta,2026-01-12
P009,Elevated CEA levels,Dr. Srinivasan,2026-01-08
P010,No recurrence after 2 years,Dr. Joshi,2025-12-25


In [0]:
schema_df = spark.read.json("/Volumes/rwd/bronze/myvolume/bronze/observation_raw/schema/*")
display(schema_df)


_corrupt_record,dataSchemaJson,partitionSchemaJson
v1,null,null
null,"{""type"":""struct"",""fields"":[{""name"":""Patient_ID"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Observation"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Doctor"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Observation_Date"",""type"":""date"",""nullable"":true,""metadata"":{""__detected_date_formats"":""yyyy-M-d""}}]}","{""type"":""struct"",""fields"":[]}"


In [0]:
%sql
MERGE INTO rwd.bronze.observation AS target
USING rwd.bronze.observation_raw AS source
ON target.patient_id = source.patient_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,0,0,10


In [0]:
%sql
select * from rwd.bronze.observation

Patient_ID,Observation,Doctor,Observation_Date
P001,Elevated tumor markers,Dr. Menon,2026-01-15
P002,No detectable lesions,Dr. Kapoor,2025-12-20
P003,Increase in lymph node size,Dr. Reddy,2026-01-10
P004,Lesion unchanged over 6 months,Dr. Sharma,2026-01-05
P005,New hepatic metastasis,Dr. Iyer,2026-01-18
P006,Clear PET scan,Dr. Banerjee,2025-11-30
P007,Multiple pulmonary nodules,Dr. Thomas,2026-01-22
P008,Minimal residual disease,Dr. Gupta,2026-01-12
P009,Elevated CEA levels,Dr. Srinivasan,2026-01-08
P010,No recurrence after 2 years,Dr. Joshi,2025-12-25
